# Analyse ETH-USDC strategy profit calculation issues

Fix issues with bad profit calculations.


In [33]:
from tradeexecutor.utils.state_downloader import download_state
import os

url = "https://enzyme-polygon-eth-usdc.tradingstrategy.ai"
name = "enzyme-polygon-eth-usdc.json"

if not os.path.exists(f"/tmp/{name}"):
    state = download_state(url)
    state.write_json_file(f"/tmp/{name}")


In [34]:
from tradeexecutor.state.state import State

state = State.read_json_file(f"/tmp/{name}")


## Balance checks

Check for current balances and abnormal balance update events.

In [35]:
portfolio = state.portfolio
asset, exchange_rate = state.portfolio.get_default_reserve_asset()
position = portfolio.get_default_reserve_position()
print(f"Number of reserve assets: {len(portfolio.reserves)}")
print(f"Reserves: {asset} at {exchange_rate} {asset.token_symbol} / USD")
print(f"Amount: {position.get_quantity()} {asset.token_symbol}")
print(f"Equity: {position.get_total_equity()} USD")
print(f"Value: {position.get_value()} USD")
print(f"Balance updates: {position.get_base_token_balance_update_quantity()} {asset.token_symbol}")
print(f"Number of balance updates events: {len(position.balance_updates)}")
print(f"Number of events references in the portfolio overall: {len(state.sync.treasury.balance_update_refs)}")

Number of reserve assets: 1
Reserves: <USDC at 0x2791bca1f2de4661ed88a30c99a7a9449aa84174> at 1.0 USDC / USD
Amount: 5.156454136718750034958702606 USDC
Equity: 5.15645413671875 USD
Value: 5.15645413671875 USD
Balance updates: -53.65080607812499814014017827 USDC
Number of balance updates events: 35
Number of events references in the portfolio overall: 32


# Vault sync status

In [36]:
treasury = state.sync.treasury
deployment = state.sync.deployment

print(f"Vault deployment: {deployment}")
print(f"Treasury sync status: {treasury}")

Vault deployment: <Deployment chain:polygon address:0x6E321256BE0ABd2726A234E8dBFc4d3caf255AE0 name:Degen Fault I token:DEGE1>
Treasury sync status: <Treasury updated:2023-11-20 14:18:00 cycle:2023-11-20 14:18:00 block scanned:50,175,386 refs:32>


## Individual events

For the last analysed executor.

In [37]:
from tradeexecutor.analysis.position import display_reserve_position_events

df = display_reserve_position_events(position)
display(df)


,Cause,At,Quantity,Dollar value,Address,Notes
1,deposit,2023-07-04 19:07:40,7.165562,7.165562,,reinit() at block 44683317
2,correction,2023-07-13 23:47:27,-7.16556200000000021552182261,-7.165562,,Accounting correction based on the actual on-c...
3,correction,2023-07-25 10:27:30,-35.71078500000000044423131840,-35.710785,,Accounting correction based on the actual on-c...
4,correction,2023-09-18 12:37:03,-6.780573828125000014239276424,-6.780574,,Accounting correction based on the actual on-c...
5,deposit,2023-10-17 19:54:48,1,1.000000,0xEFD90a79feE79aba176A7E1B0E101193fb7d7bda,
6,redemption,2023-10-18 00:48:28,-0.999997,0.999997,0xEFD90a79feE79aba176A7E1B0E101193fb7d7bda,
7,deposit,2023-10-18 08:33:51,70,70.000000,0x57c111CBc7EEb27497277D165e095E44C5B64076,
8,deposit,2023-10-18 12:07:21,10,10.000000,0x575A8d31ab7B82B327cB8e994f8d898624D9Cc3c,
9,redemption,2023-10-18 12:08:55,-69.999855,69.999855,0x57c111CBc7EEb27497277D165e095E44C5B64076,
10,deposit,2023-10-18 12:09:49,10,10.000000,0x57c111CBc7EEb27497277D165e095E44C5B64076,


## Position profit history

In [38]:
from io import StringIO
from typing import Iterable, List
import pandas as pd

from tradeexecutor.state.position import TradingPosition
from tradeexecutor.state import position

positions = state.portfolio.closed_positions.values()

data = [(p.closed_at, p.position_id, p.get_realised_profit_percent() * 100, p.is_repaired()) for p in positions]

df = pd.DataFrame(data, columns=("date", "position id", "position profit %", "failed trades"))
df = df.set_index("date")

df = df.sort_values("position profit %", ascending=True)

pd.set_option('display.max_rows', 100)

df


,position id,position profit %,failed trades
date,,,
2023-11-02 14:05:02,22,-64.368452,False
2023-10-30 18:05:35,21,-48.749665,False
2023-09-12 09:39:08,16,-4.199076,True
2023-11-10 12:21:15,27,-2.146519,False
2023-10-26 11:51:24,20,-2.117160,False
2023-11-13 21:15:16,28,-2.009435,False
2023-11-09 16:15:16,25,-1.813267,False
2023-09-06 19:01:04,15,-1.537973,False
2023-07-12 23:04:40,3,-1.202649,False


## Check prolematic position

In [40]:
p: TradingPosition
p = state.portfolio.closed_positions[22]
print("Position", p)
print("Realised profit", p.get_realised_profit_percent())
print("Avg buy", p.get_average_buy())
print("Avg sell", p.get_average_sell())
print("Buy value", p.get_buy_value())
print("Sell value", p.get_sell_value())
print("Balance updates")
for id, bu in p.balance_updates.items():
    print(bu)

Position <Closed position #22 <Pair WETH-USDC spot_market_hold at 0x45dda9cb7c25131df268515131f647d726f50608 (0.0500% fee) on exchange 0x1f98431c8ad98523631ae4a59f267346ea31f984> $14.087609495767193>
Realised profit -0.6436845170373768
Avg buy 1852.935512317986
Avg sell 1826.4457494181925
Buy value 14.087609495767193
Sell value 5.019633381273124
Balance updates
Funding event #32 redemption -0.004854553252186546 tokens for position #22 at block 49,443,918 from 0x57c111CBc7EEb27497277D165e095E44C5B64076
